# <center>Sentiment Analysis  <span style='color:Purple'>  </span>

###  1: Importing Libraries

In [28]:
import nltk
import re
import numpy as np
import pandas as pd
import gensim
import gensim.corpora as corpora

from nltk        import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
from operator    import itemgetter
from sklearn.metrics import f1_score, accuracy_score
from gensim.utils import simple_preprocess
from pprint import pprint

nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

# For max displays
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes             import MultinomialNB
from sklearn.pipeline                import Pipeline

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import classification_report

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/noopur/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/noopur/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


### 2: 
#### Read dataset  
#### Perfom Text processing for the tweets 
  * Remove Stop words
  * special characters  
  * convert the text to lowercase

In [2]:
import pandas as pd
df = pd.read_csv('Corona_NLP_train.csv', encoding='latin1')

In [3]:
df.head(10)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate supplies of regular meds but not over order,Positive
2,3801,48753,Vagabonds,16-03-2020,"Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P",Positive
3,3802,48754,NaN,16-03-2020,"My food stock is not the only one which is empty...\r\r\n\r\r\nPLEASE, don't panic, THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need. \r\r\nStay calm, stay safe.\r\r\n\r\r\n#COVID19france #COVID_19 #COVID19 #coronavirus #confinement #Confinementotal #ConfinementGeneral https://t.co/zrlG0Z520j",Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COVID19 outbreak.\r\r\n\r\r\nNot because I'm paranoid, but because my food stock is litteraly empty. The #coronavirus is a serious thing, but please, don't panic. It causes shortage...\r\r\n\r\r\n#CoronavirusFrance #restezchezvous #StayAtHome #confinement https://t.co/usmuaLq72n",Extremely Negative
5,3804,48756,"ÃT: 36.319708,-82.363649",16-03-2020,"As news of the regionÂs first confirmed COVID-19 case came out of Sullivan County last week, people flocked to area stores to purchase cleaning supplies, hand sanitizer, food, toilet paper and other goods, @Tim_Dodson reports https://t.co/cfXch7a2lU",Positive
6,3805,48757,"35.926541,-78.753267",16-03-2020,"Cashier at grocery store was sharing his insights on #Covid_19 To prove his credibility he commented ""I'm in Civics class so I know what I'm talking about"". https://t.co/ieFDNeHgDO",Positive
7,3806,48758,Austria,16-03-2020,Was at the supermarket today. Didn't buy toilet paper. #Rebel\r\r\n\r\r\n#toiletpapercrisis #covid_19 https://t.co/eVXkQLIdAZ,Neutral
8,3807,48759,"Atlanta, GA USA",16-03-2020,"Due to COVID-19 our retail store and classroom in Atlanta will not be open for walk-in business or classes for the next two weeks, beginning Monday, March 16. We will continue to process online and phone orders as normal! Thank you for your understanding! https://t.co/kw91zJ5O5i",Positive
9,3808,48760,"BHAVNAGAR,GUJRAT",16-03-2020,"For corona prevention,we should stop to buy things with the cash and should use online payment methods because corona can spread through the notes. Also we should prefer online shopping from our home. It's time to fight against COVID 19?. #govindia #IndiaFightsCorona",Negative


In [4]:
df.drop(['UserName','ScreenName','Location','TweetAt'], axis = 1, inplace = True)

In [5]:
df.head(5)

,OriginalTweet,Sentiment
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8,Neutral
1,advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate supplies of regular meds but not over order,Positive
2,"Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P",Positive
3,"My food stock is not the only one which is empty...\r\r\n\r\r\nPLEASE, don't panic, THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need. \r\r\nStay calm, stay safe.\r\r\n\r\r\n#COVID19france #COVID_19 #COVID19 #coronavirus #confinement #Confinementotal #ConfinementGeneral https://t.co/zrlG0Z520j",Positive
4,"Me, ready to go at supermarket during the #COVID19 outbreak.\r\r\n\r\r\nNot because I'm paranoid, but because my food stock is litteraly empty. The #coronavirus is a serious thing, but please, don't panic. It causes shortage...\r\r\n\r\r\n#CoronavirusFrance #restezchezvous #StayAtHome #confinement https://t.co/usmuaLq72n",Extremely Negative


In [6]:
df.shape

(41157, 2)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41157 entries, 0 to 41156
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   OriginalTweet  41157 non-null  object
 1   Sentiment      41157 non-null  object
dtypes: object(2)
memory usage: 643.2+ KB


In [8]:
df.duplicated().sum()

0

In [9]:
stop = stopwords.words('english')

In [10]:
for i in stop :
    df = df.replace(to_replace=r'\b%s\b'%i, value="",regex=True)

df.head(15)

,OriginalTweet,Sentiment
0,@MeNyrbie @Phil_Gahan @Chrisitv https://.co/iFz9FAn2Pa https://.co/xX6ghGFzCC https://.co/I2NlzdxNo8,Neutral
1,advice Talk neighbours family exchange phone numbers create contact list phone numbers neighbours schools employer chemist GP set online shopping accounts poss adequate supplies regular meds order,Positive
2,"Coronavirus Australia: Woolworths give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://.co/bInCA9Vp8P",Positive
3,"My food stock one empty...\r\r\n\r\r\nPLEASE, ' panic, THERE WILL BE ENOUGH FOOD FOR EVERYONE take need. \r\r\nStay calm, stay safe.\r\r\n\r\r\n#COVID19france #COVID_19 #COVID19 #coronavirus #confinement #Confinementotal #ConfinementGeneral https://.co/zrlG0Z520j",Positive
4,"Me, ready go supermarket #COVID19 outbreak.\r\r\n\r\r\nNot I' paranoid, food stock litteraly empty. The #coronavirus serious thing, please, ' panic. It causes shortage...\r\r\n\r\r\n#CoronavirusFrance #restezchezvous #StayAtHome #confinement https://.co/usmuaLq72n",Extremely Negative
5,"As news regionÂ first confirmed COVID-19 case came Sullivan County last week, people flocked area stores purchase cleaning supplies, hand sanitizer, food, toilet paper goods, @Tim_Dodson reports https://.co/cfXch7a2lU",Positive
6,"Cashier grocery store sharing insights #Covid_19 To prove credibility commented ""I' Civics class I know I' talking "". https://.co/ieFDNeHgDO",Positive
7,Was supermarket today. Didn' buy toilet paper. #Rebel\r\r\n\r\r\n#toiletpapercrisis #covid_19 https://.co/eVXkQLIdAZ,Neutral
8,"Due COVID-19 retail store classroom Atlanta open walk- business classes next two weeks, beginning Monday, March 16. We continue process online phone orders normal! Thank understanding! https://.co/kw91zJ5O5i",Positive
9,"For corona prevention, stop buy things cash use online payment methods corona spread notes. Also prefer online shopping home. It' time fight COVID 19?. #govindia #IndiaFightsCorona",Negative


In [11]:
df = df.replace(r'[^0-9a-zA-Z ]', '', regex=True)
df.head(15)

,OriginalTweet,Sentiment
0,MeNyrbie PhilGahan Chrisitv httpscoiFz9FAn2Pa httpscoxX6ghGFzCC httpscoI2NlzdxNo8,Neutral
1,advice Talk neighbours family exchange phone numbers create contact list phone numbers neighbours schools employer chemist GP set online shopping accounts poss adequate supplies regular meds order,Positive
2,Coronavirus Australia Woolworths give elderly disabled dedicated shopping hours amid COVID19 outbreak httpscobInCA9Vp8P,Positive
3,My food stock one emptyPLEASE panic THERE WILL BE ENOUGH FOOD FOR EVERYONE take need Stay calm stay safeCOVID19france COVID19 COVID19 coronavirus confinement Confinementotal ConfinementGeneral httpscozrlG0Z520j,Positive
4,Me ready go supermarket COVID19 outbreakNot I paranoid food stock litteraly empty The coronavirus serious thing please panic It causes shortageCoronavirusFrance restezchezvous StayAtHome confinement httpscousmuaLq72n,Extremely Negative
5,As news region first confirmed COVID19 case came Sullivan County last week people flocked area stores purchase cleaning supplies hand sanitizer food toilet paper goods TimDodson reports httpscocfXch7a2lU,Positive
6,Cashier grocery store sharing insights Covid19 To prove credibility commented I Civics class I know I talking httpscoieFDNeHgDO,Positive
7,Was supermarket today Didn buy toilet paper Rebeltoiletpapercrisis covid19 httpscoeVXkQLIdAZ,Neutral
8,Due COVID19 retail store classroom Atlanta open walk business classes next two weeks beginning Monday March 16 We continue process online phone orders normal Thank understanding httpscokw91zJ5O5i,Positive
9,For corona prevention stop buy things cash use online payment methods corona spread notes Also prefer online shopping home It time fight COVID 19 govindia IndiaFightsCorona,Negative


In [12]:
df = df.applymap(lambda s:s.lower() if type(s) == str else s)
df.head(15)

,OriginalTweet,Sentiment
0,menyrbie philgahan chrisitv httpscoifz9fan2pa httpscoxx6ghgfzcc httpscoi2nlzdxno8,neutral
1,advice talk neighbours family exchange phone numbers create contact list phone numbers neighbours schools employer chemist gp set online shopping accounts poss adequate supplies regular meds order,positive
2,coronavirus australia woolworths give elderly disabled dedicated shopping hours amid covid19 outbreak httpscobinca9vp8p,positive
3,my food stock one emptyplease panic there will be enough food for everyone take need stay calm stay safecovid19france covid19 covid19 coronavirus confinement confinementotal confinementgeneral httpscozrlg0z520j,positive
4,me ready go supermarket covid19 outbreaknot i paranoid food stock litteraly empty the coronavirus serious thing please panic it causes shortagecoronavirusfrance restezchezvous stayathome confinement httpscousmualq72n,extremely negative
5,as news region first confirmed covid19 case came sullivan county last week people flocked area stores purchase cleaning supplies hand sanitizer food toilet paper goods timdodson reports httpscocfxch7a2lu,positive
6,cashier grocery store sharing insights covid19 to prove credibility commented i civics class i know i talking httpscoiefdnehgdo,positive
7,was supermarket today didn buy toilet paper rebeltoiletpapercrisis covid19 httpscoevxkqlidaz,neutral
8,due covid19 retail store classroom atlanta open walk business classes next two weeks beginning monday march 16 we continue process online phone orders normal thank understanding httpscokw91zj5o5i,positive
9,for corona prevention stop buy things cash use online payment methods corona spread notes also prefer online shopping home it time fight covid 19 govindia indiafightscorona,negative


### 3: Split train and test dataset

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
df1 = df.copy()

In [15]:
X = df1.OriginalTweet
y = df1.Sentiment
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42) 

In [16]:
# X = df1.drop('Sentiment', axis=1)
# y = df1[["Sentiment"]]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42) 

In [17]:
X_train.shape

(28809,)

In [18]:
y_train.shape

(28809,)

### 4: Create pipeline and define parameters for GridSearch

In [19]:
text_clf = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('clf', MultinomialNB())])


In [20]:
tuned_parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': [1, 1e-1, 1e-2]
}

### 5: classification (using GridSearch) 

In [30]:
clf1 = GridSearchCV(text_clf, tuned_parameters, cv=5, n_jobs=-1, verbose=1, scoring='accuracy')
clf1.fit(X_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   51.8s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  3.3min finished


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', MultinomialNB())]),
             n_jobs=-1,
             param_grid={'clf__alpha': [1, 0.1, 0.01],
                         'tfidf__norm': ('l1', 'l2'),
                         'tfidf__use_idf': (True, False),
                         'vect__ngram_range': [(1, 1), (1, 2), (2, 2)]},
             scoring='accuracy', verbose=1)

### 6: confusion matrix, accuracy, F1 score on the test dataset

In [26]:
y_pred = clf.predict(X_test)
pd.crosstab(y_test,y_pred,rownames=['Predicted'],colnames=['True'],margins=True)

True,extremely negative,extremely positive,negative,neutral,positive,All
Predicted,,,,,,
extremely negative,592,22,656,50,252,1572
extremely positive,25,820,181,49,914,1989
negative,369,135,1398,201,902,3005
neutral,83,100,477,883,749,2292
positive,103,505,763,261,1858,3490
All,1172,1582,3475,1444,4675,12348


In [25]:
print("Detailed classification report:")
print(classification_report(y_test, clf.predict(X_test), digits=4))

Detailed classification report:
                    precision    recall  f1-score   support

extremely negative     0.5051    0.3766    0.4315      1572
extremely positive     0.5183    0.4123    0.4593      1989
          negative     0.4023    0.4652    0.4315      3005
           neutral     0.6115    0.3853    0.4727      2292
          positive     0.3974    0.5324    0.4551      3490

          accuracy                         0.4495     12348
         macro avg     0.4869    0.4343    0.4500     12348
      weighted avg     0.4715    0.4495    0.4503     12348



In [29]:
print('Accuracy :', accuracy_score(y_test, y_pred))
print('f1 score :', f1_score(y_test, y_pred, average='macro'))

Accuracy : 0.44954648526077096
f1 score : 0.45000696675851515
